In [1]:
#pragma cling add_include_path("/home/slava/itensor/")
#pragma cling add_library_path("/home/slava/itensor/lib")
#pragma cling load("libitensor.so")
#pragma cling load("libitensor-g.so")
#pragma cling load("/home/slava/itensor/itensor/all_mps.h")
using namespace itensor;
#include "xtensor/xarray.hpp"


using namespace xt;
using namespace std;

// #pragma cling add_include_path("/home/slava/boost/")
// #pragma cling add_library_path("/home/slava/boost/lib")

// #include "boost/any.hpp"

using array_type = xt::xarray<double, xt::layout_type::row_major>;
using shape_type = array_type::shape_type;

In [2]:
int N = 10;
SpinHalf sites(N);
AutoMPO ampo(sites);

In [2]:
char* op1, op2;

In [3]:
op1 = "S+";

input_line_12:2:8: warning: ISO C++11 does not allow conversion from string literal to 'char *' [-Wwritable-strings]
 op1 = "S+";
       ^


In [4]:
char op2[] = "S+";

In [5]:
char op2[] = "S+";
int ind1 = 1;
int ind2 = 2;
char op1[] = "S+";
Complex coef = 1;

In [6]:
Complex coef = 1;

In [10]:
ampo += coef,op1,ind1,op2,ind2;

In [5]:
ampo += {0.5,"S+",1,"S-",2};

input_line_14:2:7: error: no viable overloaded '+='
 ampo += {0.5,"S+",1,"S-",2};
 ~~~~ ^  ~~~~~~~~~~~~~~~~~~~
/home/slava/itensor/itensor/qn.h:229:1: note: candidate function not viable: no known conversion from 'itensor::AutoMPO' to 'itensor::QNVal &' for 1st argument
operator+=(QNVal& qva, QNVal const& qvb);
^
/home/slava/itensor/itensor/qn.h:274:1: note: candidate function not viable: no known conversion from 'itensor::AutoMPO' to 'itensor::QN &' for 1st argument
operator+=(QN & qa, QN const& qb);
^
/home/slava/itensor/itensor/tensor/vec.h:66:1: note: candidate function not viable: no known conversion from 'itensor::AutoMPO' to 'itensor::VectorRef' (aka
      'itensor::TenRef<itensor::VecRangeT<0>, double>') for 1st argument
operator+=(VectorRef a, VectorRefc b);
^
/home/slava/itensor/itensor/tensor/vec.h:94:1: note: candidate function not viable: no known conversion from 'itensor::AutoMPO' to 'itensor::Vector &' (aka
      'itensor::Ten<itensor::VecRangeT<0>, double> &') for 1st a

Interpreter Error: 

In [2]:
int N = 10;
SpinHalf sites(N);
AutoMPO ampo(sites);
for(int j = 1; j < N;++j)
  {
  ampo += 0.5,"S+",j,"S-",j+1;
  ampo += 0.5,"S-",j,"S+",j+1;
  ampo +=     "Sz",j,"Sz",j+1;
  }
auto H = MPO(ampo);

In [4]:
args.insert(args.begin(), 1);

In [5]:
args.insert(args.begin(),"I");

In [6]:
args.insert(args.begin(), 1);

In [7]:
args

{ @0x5569c91f33c0, @0x5569c9194aa0, @0x5569c7bf4360 }

In [2]:
int N_mode = 4;
auto sites = SpinHalf(N_mode);
auto ampo = AutoMPO(sites);

In [3]:
ampo += 1,"Sx",2;

In [5]:
auto mpo = MPO(ampo);

In [2]:
struct RetValH {
  SpinHalf sites;
  MPO H;
};
struct RetValDMRG {
  SpinHalf sites;
  MPS psi;
};
Args::global();

In [3]:
RetValH get_H_SSH(int const& N_mode, Args const& args = Args::global())
{   
    char op1[] = "Sx";
    char op2[] = "Sy";
    int ind1 = 1;
    int ind2 = 2;
    Complex coef = 1;
    
    float t_p = args.getReal("t_p",1);
    float t_m = args.getReal("t_m",0);
    float B = args.getReal("B",0);
    auto sites = SpinHalf(N_mode);

    auto ampo = AutoMPO(sites);
    for(int j = 1; j < N_mode; ++j)
        {
        coef = 4*(t_p + pow(-1,j-1)*t_m);
        ampo += coef,op1,j,op1,j+1;
//         ampo += coef,op1,ind1,op2,ind2;
        ampo += coef,op2,j,op2,j+1;
//         ampo += coef,"Sx",j;
        }
    
    if (B !=0 )
        {
        ampo += B,"Sz",1;
        ampo += B,"Sz",N_mode;
        }
    return {sites, MPO(ampo)};
}

In [4]:
RetValDMRG dmrg_ssh(int const& N_mode, Args const& args = Args::global())
{   
    auto ret_val_H = get_H_SSH(N_mode, args);
    auto sites = ret_val_H.sites;

    auto H = ret_val_H.H;

    //Set up random initial wavefunction
    auto psi = MPS(sites);

    //Perform 5 sweeps of DMRG
    auto sweeps = Sweeps(5);
    //Specify max number of states kept each sweep
    sweeps.maxm() = 50, 50, 100, 100, 200;

    dmrg(psi,H,sweeps,{"Silent",true});

    //Continue to analyze wavefunction afterward 
//     Real energy = overlap(psi,H,psi);

    return {sites, psi};
}

In [5]:
array_type get_correlators(MPS psi, SiteSet sites, std::string op_name)
{   
    ITensor C, C_j;

    int N_mode = sites.N();
    
    size_t N_C = (N_mode*(N_mode-1))/2;
//     static float correlators[N_C] = {};
    shape_type shape = {N_C};
    array_type correlators(shape);
    
    int k = 0;
    for(int i = 1; i < N_mode; ++i){

        psi.position(i);
        auto S_i = sites.op(op_name,i);


        // Calculate first correlator at site i
        C = psi.A(i);
        C *= S_i;
        auto ir = commonIndex(psi.A(i),psi.A(i+1),Link);
        C *= dag(prime(prime(psi.A(i),Site),ir));

        for(int j = i+1; j <= N_mode; ++j){

            // Common operation
            C *= psi.A(j);

            // Calculate second correlator at site j
            auto S_j = sites.op(op_name,j);
            C_j = C*S_j;
            auto il = commonIndex(psi.A(j),psi.A(j-1),Link);
            C_j *=  dag(prime(prime(psi.A(j),Site),il));

            correlators(k) = 4*C_j.real();
//             println(correlators(k));
            k+=1;
    //         // Continue contraction
            C *= dag(prime(psi.A(j),Link));    
            }
        }
    
    return correlators;
}

In [6]:
array_type get_SSH_correlators(int const& N_mode, Args const& args = Args::global())
{   
    auto ret_val_dmrg = dmrg_ssh(N_mode, args);
    MPS psi = ret_val_dmrg.psi;
    auto sites = ret_val_dmrg.sites;

    array_type correlators_x = get_correlators(psi, sites, "Sx");
    array_type correlators_y = get_correlators(psi, sites, "Sy");
    
    int N_C = (N_mode*(N_mode-1))/2;
    for(int j = 0; j < N_C; ++j){
        correlators_x(j) += correlators_y(j);
    }
//     for(int j = 1; j <= N_C; ++j){
//         println(correlators_x(j));
//     }
    return correlators_x;
}

In [7]:
int N_mode = 10;

auto args = Args("t_p=",1,
                 "t_m=",0,
                 "B=",0.1);

array_type correlators = get_SSH_correlators(N_mode, args)

In [8]:
correlators(1)

0.857127

In [14]:
get_SSH_correlators(N_mode, args)

@0x5606241679f0

In [9]:
int N_C = (N_mode*(N_mode-1))/2;
for(int j = 1; j <= N_C; ++j){
    println(correlators[j]);
}

0.857127
-0.818818
0.580918
-0.580918
0.436829
-0.457192
0.318235
-0.370756
-1.00011
0.746897
-0.529885
0.507746
-0.381721
0.394076
-0.274256
0.318237
-1.49362
0.812571
-0.778547
0.557157
-0.575068
0.394078
-0.457194
-1.08799
0.793979
-0.568127
0.557156
-0.381723
0.436829
-1.45942
0.793979
-0.778547
0.50775
-0.580918
-1.08799
0.812571
-0.529884
0.580919
-1.49362
0.746897
-0.818818
-1.00011
0.857127
-1.71447
7.16395e-322


In [9]:
int N_C = (N_mode*(N_mode-1))/2;
for(int j = 1; j <= N_C; ++j){
    println(correlators[j]);
}

0.857127
-0.818818
0.580918
-0.580919
0.436829
-0.457193
0.318235
-0.370756
-1.00011
0.746896
-0.529885
0.507746
-0.381721
0.394075
-0.274255
0.318237
-1.49362
0.812571
-0.778547
0.557156
-0.575064
0.394076
-0.457194
-1.08799
0.793979
-0.568127
0.557154
-0.381722
0.436829
-1.45942
0.793979
-0.778547
0.50775
-0.580918
-1.08799
0.812571
-0.529884
0.580919
-1.49362
0.746897
-0.818818
-1.00011
0.857127
-1.71447
4.79244e-322


In [6]:
#include "xtensor/xarray.hpp"
#include <array>

using namespace xt;

In [7]:
xarray<double>::shape_type shape2 = {10};

In [8]:
xarray<double> a4(shape2);

In [32]:
a2(1) = 1;
a2(2) = 2;

In [36]:
a2(3)

1.10556e+161

In [1]:
#include <vector>
#include "xtensor/xarray.hpp"

std::vector<size_t> shape = { 3, 2, 4 };
xt::xarray<double, xt::layout_type::row_major> a(shape);

In [7]:
a[2]

-nan

In [1]:
#include <vector>

In [2]:
using namespace std;


In [3]:
static int m3 = 3

In [4]:
std::vector<double> a(m3);

In [11]:
println(a[1])

input_line_22:2:2: error: use of undeclared identifier 'println'
 println(a[1])
 ^


Interpreter Error: 

In [9]:
a[1]

0

In [49]:
a3[0] = 1;
a3[1] = 2;
a3[8] = 3;
a3[12] = 4;

In [54]:
a3[11]

4.00193e-322

In [3]:
std::vector<double> f(){
    vector<double> out(10);
    return out;
}

input_line_9:2:5: error: no template named 'vector'; did you mean 'std::vector'?
    vector<double> out(10);
    ^~~~~~
    std::vector
/home/slava/miniconda3/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/bits/stl_vector.h:216:11: note: 'std::vector' declared here
    class vector : protected _Vector_base<_Tp, _Alloc>
          ^


Interpreter Error: 

In [57]:
vector<double> x = f()

In [63]:
x[9] = 10

10

In [1]:
int x = 10

In [2]:
delete x

input_line_8:2:2: error: cannot delete expression of type 'int'
 delete x
 ^      ~


Interpreter Error: 